In [ ]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
headers = {
    "cookie": "AWSALB=C32%2BQz6UaMr0oSv2aIORy%2BFDkIwMuHtCG0NXsBrzoJ%2BYRd0Det3ivYb0ar756gXna7wkAr6K%2FPSGReZbkPPgvR0WYrNBhquF90h5vQV3kbAbo6URXbAOFkYJI42e; AWSALBCORS=C32%2BQz6UaMr0oSv2aIORy%2BFDkIwMuHtCG0NXsBrzoJ%2BYRd0Det3ivYb0ar756gXna7wkAr6K%2FPSGReZbkPPgvR0WYrNBhquF90h5vQV3kbAbo6URXbAOFkYJI42e",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": "https://fairfax.usedirect.com",
    "Connection": "keep-alive",
    "Referer": "https://fairfax.usedirect.com/FairfaxFCPAWeb/ACTIVITIES/Search.aspx?category_name=CAMPS&place_id=ALL+PLACES%2fpage-3",
    "Cookie": "AWSALB=D9nq3VUl6ofSKBWNyTpiWbefCD57uCGj2csysK1Htr1WXj2wGFjNrPq1I11zdKhWHFY7RBdlPNp7kKruAjV9cXkVQkTl1za1rX2hTVR0US/cbAGLppwMkOJPcynC; AWSALBCORS=D9nq3VUl6ofSKBWNyTpiWbefCD57uCGj2csysK1Htr1WXj2wGFjNrPq1I11zdKhWHFY7RBdlPNp7kKruAjV9cXkVQkTl1za1rX2hTVR0US/cbAGLppwMkOJPcynC; ASP.NET_SessionId=4vdlzw2vszbzg5ano10g3vqq; __AntiXsrfToken=8168851a341e4658bc69666d2ff7b19f; _ga=GA1.2.1404541115.1642648378; _gid=GA1.2.1361117984.1642648378",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1"
}
url = 'https://fairfax.usedirect.com/FairfaxFCPAWeb/ACTIVITIES/Search.aspx?category_name=CAMPS&place_id=ALL+PLACES'
def get_url(url):
  r = requests.get(url,headers=headers)
  soup = BeautifulSoup(r.content,'html.parser')
  s = requests.session()
  VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
  VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
  EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']
  bNext = dict(__EVENTTARGET="ctl01$mainContent$bNext", __EVENTARGUMENT="",  __VIEWSTATE=VIEWSTATE, __VIEWSTATEGENERATOR=VIEWSTATEGENERATOR, __EVENTVALIDATION=EVENTVALIDATION)
  next = s.post(url, data=bNext,headers=headers)
  soup1=BeautifulSoup(next.content,'html.parser')


In [176]:
data  =[]
seat  =[]
title =[]
Head  =[]
for i in range(1,25):
  get_url(url)
  layer = soup.find_all('div', {'id':'activityList'})
  row   = layer[0].find_all('tr')
  for row in row:
    h2  = row.find_previous('h2',{'class':'FindTour'}, text=None).text
    d1  = row.find_all('td',{'style':"width: 15%"})
    d2  = row.find_all('td',{'style':"width: 6%"})
    h3 =["".join(h2)]
    data.append([ele.text.strip() for ele in d1 if ele]) # Get rid of empty values
    seat.append([ele.text.strip() for ele in d2 if ele])    
    title.append([ele for ele in h3 if ele])



In [208]:
df = pd.DataFrame(data,columns=['Location', 'Code', 'Time'])
df["seats"]= seat 
df["Activity"]= title
df = df.dropna()
out=df['seats'].str[0]
out2=df['Activity'].str[0]
df.drop(['seats', 'Activity'], axis=1)
df["seats"]= out
df["Activity"]=out2
clean = df.replace('None',0 )
clean.to_excel('Camp.xlsx')

In [207]:
clean


,Location,Code,Time,seats,Activity
1,YKG.NVS1,Virtual FCPA,06/20/2022 - 06/24/2022\rWeekdays : 1:00 PM - ...,19,3D Game Design with Unity (11-14 yrs.)\r\n
3,6BP.97N7,Virtual FCPA,06/20/2022 - 06/24/2022\rWeekdays : 9:30 AM - ...,19,3D Game Design with Unity (8-11 yrs.)\r\n
5,IWQ.9JO5,Audrey Moore Rec Center,06/21/2022 - 06/24/2022\rTue Wed Thu Fri : 9:0...,0,Abrakadoodle Art Madness (6-12 yrs)\r\n
6,IWQ.Y3N7,Providence Rec Center,06/27/2022 - 07/01/2022\rWeekdays : 9:00 AM - ...,5,Abrakadoodle Art Madness (6-12 yrs)\r\n
7,IWQ.TAOU,Spring Hill Rec Center,07/05/2022 - 07/08/2022\rTue Wed Thu Fri : 9:0...,6,Abrakadoodle Art Madness (6-12 yrs)\r\n
...,...,...,...,...,...
1577,B2C.58LH,Oak Marr Rec Center,06/27/2022 - 07/01/2022\rWeekdays : 9:00 AM - ...,0,Adventure Fishing Camp (9-14 yrs.) \r\n
1578,B2C.XTHT,Providence Rec Center,07/11/2022 - 07/15/2022\rWeekdays : 9:00 AM - ...,0,Adventure Fishing Camp (9-14 yrs.) \r\n
1580,JXL.0EJ0,Hidden Pond Nature Center,07/11/2022 - 07/15/2022\rWeekdays : 9:00 AM - ...,0,Adventures in Geology & Hydrology (8-11yr)\r\n
1581,JXL.Q3KG,Hidden Pond Nature Center,08/08/2022 - 08/12/2022\rWeekdays : 9:00 AM - ...,0,Adventures in Geology & Hydrology (8-11yr)\r\n
